In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pdb
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
!pip install pytorch-pretrained-bert

You are using pip version 18.0, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import numpy as np

In [4]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [10]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Lin

In [11]:
# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)

NameError: name 'tokens_tensor' is not defined

## Load Data

In [12]:
file_name = '/Users/avi/Dropbox/Insight/Project/insight_project/data/interim/charity_data_cleaned.csv'
all_charity = pd.read_csv(file_name)

In [13]:
all_charity['description_noname'] = all_charity.apply(lambda x: x['description'].replace(x['name'],""),axis=1)

In [14]:
def docs_2_bert(docs):
    
    bert_vecs = []
    idx = 0
    for doc in docs:
        print(idx, end = '')
        marked_text = "[CLS] " + doc + " [SEP]"

        tokenized_text = tokenizer.tokenize(marked_text)

        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

        segments_ids = [1] * len(tokenized_text)

        # Convert inputs to PyTorch tensors
        tokens_tensor = torch.tensor([indexed_tokens])

        segments_tensors = torch.tensor([segments_ids])

        # Predict hidden states features for each layer
        with torch.no_grad():
            encoded_layers, _ = model(tokens_tensor, segments_tensors)
    
        bert_vecs.append(torch.mean(encoded_layers[11], 1).reshape(1,-1))
        idx = idx + 1
    
    return bert_vecs

In [15]:
mission_text = all_charity['description_noname'].astype('str')

#Preprocess mission descriptions
mission_text_bert = docs_2_bert(mission_text)

0123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100101102103104105106107108109110111112113114115116117118119120121122123124125126127128129130131132133134135136137138139140141142143144145146147148149150151152153154155156157158159160161162163164165166167168169170171172173174175176177178179180181182183184185186187188189190191192193194195196197198199200201202203204205206207208209210211212213214215216217218219220221222223224225226227228229230231232233234235236237238239240241242243244245246247248249250251252253254255256257258259260261262263264265266267268269270271272273274275276277278279280281282283284285286287288289290291292293294295296297298299300301302303304305306307308309310311312313314315316317318319320321322323324325326327328329330331332333334335336337338339340341342343344345346347348349350351352353354355356357358359360361362363364365366367368369

2326232723282329233023312332233323342335233623372338233923402341234223432344234523462347234823492350235123522353235423552356235723582359236023612362236323642365236623672368236923702371237223732374237523762377237823792380238123822383238423852386238723882389239023912392239323942395239623972398239924002401240224032404240524062407240824092410241124122413241424152416241724182419242024212422242324242425242624272428242924302431243224332434243524362437243824392440244124422443244424452446244724482449245024512452245324542455245624572458245924602461246224632464246524662467246824692470247124722473247424752476247724782479248024812482248324842485248624872488248924902491249224932494249524962497249824992500250125022503250425052506250725082509251025112512251325142515251625172518251925202521252225232524252525262527252825292530253125322533253425352536253725382539254025412542254325442545254625472548254925502551255225532554255525562557255825592560256125622563256425652566256725682569257025712572257325742575

4375437643774378437943804381438243834384438543864387438843894390439143924393439443954396439743984399440044014402440344044405440644074408440944104411441244134414441544164417441844194420442144224423442444254426442744284429443044314432443344344435443644374438443944404441444244434444444544464447444844494450445144524453445444554456445744584459446044614462446344644465446644674468446944704471447244734474447544764477447844794480448144824483448444854486448744884489449044914492449344944495449644974498449945004501450245034504450545064507450845094510451145124513451445154516451745184519452045214522452345244525452645274528452945304531453245334534453545364537453845394540454145424543454445454546454745484549455045514552455345544555455645574558455945604561456245634564456545664567456845694570457145724573457445754576457745784579458045814582458345844585458645874588458945904591459245934594459545964597459845994600460146024603460446054606460746084609461046114612461346144615461646174618461946204621462246234624

In [16]:
charity_docs_emb = np.vstack(mission_text_bert)

np.save('bert_vecs', charity_docs_emb)

In [3]:
charity_docs_emb = np.load('bert_vecs.npy')

In [17]:
np.shape(charity_docs_emb)

(5637, 768)

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
def compute_similarity_output_n(art_emb,charity_docs_emb,topn):
    
    #compute cosine distance from article embedding to all charities
    sim_to_charities = cosine_similarity(art_emb,charity_docs_emb)
    
    #find topN similarity scores
    sim_scores_sorted = -np.sort(-sim_to_charities).flatten()
    topN_scores = sim_scores_sorted[:topn]
    
    #find topN indices
    indices_sorted = (-sim_to_charities).argsort().flatten()
    topN_indices = indices_sorted[:topn].flatten()
    
    return topN_scores, topN_indices, sim_to_charities

In [20]:
def topN_ranked_charities(charity_df, topN_scores, topN_indices):
    
    charity_df_slim = charity_df[['name','subcategory','category','score','description']]
    
    #Extract topN charities and info
    similar_charities = charity_df_slim.iloc[topN_indices].reset_index()
    
    #Add their similarity scores
    similar_charities['sim_score'] = topN_scores
    
    return similar_charities

In [26]:
text= 'How big was the global climate strike? 4 million people, activists estimate. It was likely the largest climate protest in world history.'

In [27]:
art_emb = docs_2_bert([text])

0

In [28]:
art_emb = np.vstack(art_emb)

In [29]:
topN_scores, topN_indices, sim_scores = compute_similarity_output_n(art_emb,charity_docs_emb,10)
topN_charities = topN_ranked_charities(all_charity, topN_scores, topN_indices)

In [30]:
topN_charities

,index,name,subcategory,category,score,description,sim_score
0,3227,Action Against Hunger USA,Development and Relief Services,International,93.98,Action Against Hunger is the world's hunger sp...,0.777651
1,4441,Do Something,"Youth Development, Shelter, and Crisis Services",Human Services,93.37,DoSomething.org is the largest tech company ex...,0.770378
2,4111,Chesapeake Climate Action Network,Environmental Protection and Conservation,Environment,91.29,The Chesapeake Climate Action Network (CCAN) i...,0.770154
3,2787,Water for Good,Development and Relief Services,International,83.60,Water for Good is tackling water poverty in on...,0.767456
4,4807,Food Lifeline,"Food Banks, Food Pantries, and Food Distribution",Human Services,91.13,Food Lifeline has been working to end hunger i...,0.756933
5,1957,Special Olympics Missouri,Multipurpose Human Service Organizations,Human Services,88.50,Special Olympics transforms lives through the ...,0.756642
6,2258,Samaritan's Feet,Development and Relief Services,International,92.38,Samaritan's Feet is a 501(c)(3) humanitarian a...,0.755445
7,5122,Global Hope Network International,Development and Relief Services,International,89.58,"The vast, hidden majority of the world's poore...",0.755392
8,3548,Book Trust,Youth Education Programs and Services,Education,89.78,"At Book Trust, all our work revolves around a ...",0.752985
9,1357,Pure Earth,Environmental Protection and Conservation,Environment,97.74,"Formerly known as Blacksmith Institute, Pure E...",0.752841


In [106]:
topN_charities['description'][0]

"Founded in 1968 Action on Smoking and Health (ASH) is a national legal-action antismoking and nonsmokers' rights organization which helped ban cigarette commercials, started the modern nonsmokers' rights movement, pioneered using legal action against smoking, developed novel legal theories to protect children from tobacco smoke at home and in cars, get lower health insurance rates for nonsmokers, fought tobacco subsidies, and much more."